# **Settings**

In [1]:
import os
import numpy as np 
import pandas as pd
import re
import json
import pickle
from sklearn.preprocessing import LabelEncoder

In [2]:
# encoder 저장
with open('./data2/encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

# **1. 데이터 불러오기**

# **0) 중립 불러오기**

In [25]:
rand_idx = np.random.choice(64000, 6500, replace=False)

train_origin = pd.read_csv('./data2/train.csv')
train_neutrality = train_origin[train_origin['emotion']=='Neutrality'].reset_index(drop=True)
train_neutrality = train_neutrality.iloc[rand_idx]
train_neutrality

,text,emotion,sensitivity,label
60709,넓다란 바닷가 모래밭에 누웠습니다,Neutrality,담백하다,5
31431,빵을 갖고 건물 안에 들어가는 것은 규칙 위반입니다,Neutrality,엄밀하다,5
5140,사정을 안다면 존경심이 생기겠지,Neutrality,어렴풋하다,5
57292,그 어느 날 밤 일이 문득문득 생각하였다,Neutrality,어렴풋하다,5
24529,깨끗하게 적어두겠다,Neutrality,담백하다,5
...,...,...,...,...
19851,험하게 생긴 독버섯이 있다,Neutrality,자세하다,5
63779,정신을 못 차릴 지경이었습니다,Neutrality,자세하다,5
25114,산과 바위 계곡이 있었습니다,Neutrality,뚜렷하다,5
38814,수많은 사람이 이 의원을 향하여 찾아와야 되겠다는 것이다,Neutrality,담백하다,5


In [32]:
rand_idx = np.random.choice(7966, 2000, replace=False)

val_origin = pd.read_csv('./data2/val.csv')
val_neutrality = val_origin[val_origin['emotion']=='Neutrality'].reset_index(drop=True)
val_neutrality = val_neutrality.iloc[rand_idx]
val_neutrality

,text,emotion,sensitivity,label
6232,저는 마음대로 자유스러운 음악을 개척하려 하였습니다,Neutrality,담백하다,5
4455,우리가 밥 짓고 있다,Neutrality,담백하다,5
868,유동적인 시간에 염두에 둔 것은 확실히 베르그송이즘이다,Neutrality,뚜렷하다,5
5061,그는 용감히 바닷가로 뛰어간다 이따금 뒤를 돌아보면서,Neutrality,뚜렷하다,5
7467,오늘 아침 내게 한 보고를 여기 다시 뇌이게,Neutrality,엄중하다,5
...,...,...,...,...
7575,하도 심심하여서 기차 다니는 구경이나 하는 것밖에 없다,Neutrality,예사롭다,5
5523,순결과 무한한 자유의 세상을 동경하는 마음이다,Neutrality,예사롭다,5
1192,그의 심리적 수법은 심화되었다,Neutrality,어렴풋하다,5
5981,귀엽게 굴면서 손목 잡고 한 학교에 다녔습니다,Neutrality,긴밀하다,5


# **1) Train data**

In [24]:
file_names = ['기쁨','슬픔','분노','불안','상처','당황']

print('Start!', end=' ')
data_list = []
for i, name in enumerate(file_names,1):
    print(name, end=' ')
    file_path = f"./015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training/라벨링데이터/TL4/TL4/4.감정X발화스타일/{2*i}.{name}X대화체"
    
    cnt = 0
    for folder in os.listdir(file_path):
        deep_path = os.path.join(file_path, folder)
        json_list = os.listdir(deep_path)
        for json_file in json_list:
            with open(os.path.join(deep_path,json_file),'r',encoding='utf-8') as f:
                data = json.load(f)

            text = data['전사정보']['OrgLabelText']
            emotion = data['화자정보']['Emotion']
            sensitivity = data['화자정보']['Sensitivity']
            data_list.append([text, emotion, sensitivity])
        
            cnt += 1
            if cnt % 500 == 0:
                print(f'> {cnt}', end=' ')
            
    print()

Start! 기쁨 > 500 > 1000 > 1500 > 2000 > 2500 > 3000 > 3500 > 4000 > 4500 > 5000 > 5500 > 6000 > 6500 
슬픔 > 500 > 1000 > 1500 > 2000 > 2500 > 3000 > 3500 > 4000 > 4500 > 5000 > 5500 
분노 > 500 > 1000 > 1500 > 2000 > 2500 > 3000 > 3500 > 4000 > 4500 > 5000 > 5500 > 6000 > 6500 > 7000 > 7500 
불안 > 500 > 1000 > 1500 > 2000 > 2500 > 3000 > 3500 > 4000 > 4500 > 5000 > 5500 
상처 > 500 > 1000 > 1500 > 2000 > 2500 > 3000 > 3500 > 4000 > 4500 > 5000 > 5500 
당황 > 500 > 1000 > 1500 > 2000 > 2500 > 3000 > 3500 > 4000 > 4500 > 5000 > 5500 


In [26]:
train_data = pd.DataFrame(data_list,columns=['text','emotion','sensitivity'])

train_data['label'] = encoder.transform(train_data['emotion'])
train_data = pd.concat([train_data,train_neutrality]).reset_index(drop=True)
train_data.to_csv('./data3/train_data.csv',index=False)
print(train_data.shape)
train_data

(44933, 4)


,text,emotion,sensitivity,label
0,마음의 짐을 내려놓을 수 있다.,Happy,N/A,3
1,나는 그런 소릴 듣지 않아도 된다.,Happy,N/A,3
2,썩은 달걀과 구운 닭벼슬이 가장 훌륭한 요리였답니다.,Happy,N/A,3
3,그것이 언제나 저런 목마를 타게 되나.,Happy,N/A,3
4,그와 그녀에게 있어서 글자 그대로의 첫사랑이었다.,Happy,N/A,3
...,...,...,...,...
44928,험하게 생긴 독버섯이 있다,Neutrality,자세하다,5
44929,정신을 못 차릴 지경이었습니다,Neutrality,자세하다,5
44930,산과 바위 계곡이 있었습니다,Neutrality,뚜렷하다,5
44931,수많은 사람이 이 의원을 향하여 찾아와야 되겠다는 것이다,Neutrality,담백하다,5


In [28]:
train_data['emotion'].value_counts()

emotion
Angry          7977
Happy          6500
Neutrality     6500
Anxious        5995
Sad            5991
Embarrassed    5991
Hurt           5979
Name: count, dtype: int64

# **2) Validation Data**

In [35]:
file_names = ['기쁨','슬픔','분노','불안','상처','당황']

print('Start!', end=' ')
data_list = []
for i, name in enumerate(file_names,1):
    print(name, end=' ')
    file_path = f"./015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/라벨링데이터/4.감정X발화스타일/{2*i}.{name}X대화체"
    
    cnt = 0
    for folder in os.listdir(file_path):
        deep_path = os.path.join(file_path, folder)
        json_list = os.listdir(deep_path)
        for json_file in json_list:
            with open(os.path.join(deep_path,json_file),'r',encoding='utf-8') as f:
                data = json.load(f)

            text = data['전사정보']['OrgLabelText']
            emotion = data['화자정보']['Emotion']
            sensitivity = data['화자정보']['Sensitivity']
            data_list.append([text, emotion, sensitivity])
        
            cnt += 1
            if cnt % 500 == 0:
                print(f'> {cnt}', end=' ')
            
    print()

Start! 기쁨 > 500 > 1000 > 1500 > 2000 
슬픔 > 500 > 1000 > 1500 > 2000 
분노 > 500 > 1000 > 1500 
불안 > 500 > 1000 > 1500 
상처 > 500 > 1000 > 1500 
당황 > 500 > 1000 > 1500 


In [36]:
val_data = pd.DataFrame(data_list,columns=['text','emotion','sensitivity'])
val_data['label'] = encoder.transform(val_data['emotion'])
val_data = pd.concat([val_data,val_neutrality]).reset_index(drop=True)
val_data.to_csv('./data3/val_data.csv',index=False)
print(val_data.shape)
val_data

(13985, 4)


,text,emotion,sensitivity,label
0,마음의 짐을 내려놓을 수 있다.,Happy,N/A,3
1,나는 그런 소릴 듣지 않아도 된다.,Happy,N/A,3
2,썩은 달걀과 구운 닭벼슬이 가장 훌륭한 요리였답니다.,Happy,N/A,3
3,그것이 언제나 저런 목마를 타게 되나.,Happy,N/A,3
4,그와 그녀에게 있어서 글자 그대로의 첫사랑이었다.,Happy,N/A,3
...,...,...,...,...
13980,하도 심심하여서 기차 다니는 구경이나 하는 것밖에 없다,Neutrality,예사롭다,5
13981,순결과 무한한 자유의 세상을 동경하는 마음이다,Neutrality,예사롭다,5
13982,그의 심리적 수법은 심화되었다,Neutrality,어렴풋하다,5
13983,귀엽게 굴면서 손목 잡고 한 학교에 다녔습니다,Neutrality,긴밀하다,5


In [38]:
val_data['emotion'].value_counts()

emotion
Happy          2000
Sad            2000
Neutrality     2000
Embarrassed    1998
Angry          1997
Anxious        1997
Hurt           1993
Name: count, dtype: int64

# **2. 데이터 전처리**

In [39]:
train_data = pd.read_csv('./data3/train_data.csv')
val_data = pd.read_csv('./data3/val_data.csv')
print(train_data.shape, val_data.shape)

(44933, 4) (13985, 4)


In [40]:
train_data['text'] = train_data['text'].apply(lambda x: re.sub('[^ㄱ-ㅣ가-힣\s]','',x))
val_data['text'] = val_data['text'].apply(lambda x: re.sub('[^ㄱ-ㅣ가-힣\s]','',x))

In [41]:
train_data.to_csv('./data3/train.csv',index=False)
val_data.to_csv('./data3/val.csv',index=False)